# Reading ISMN data (`ismn.interface.ISMN_Interface`)
This example shows the basic functionality to read data downloaded from the International Soil Moisture Network (ISMN).
The data can be selected and downloaded for free from <http://ismn.earth> after registration.

For this tutorial, data for ISMN networks 'REMEDHUS', 'SMOSMANIA', 'FMI', 'WEGENERNET', 'GTK', 'VAS' and 'RSMN' between 2009-08-04 and 2020-12-12 was downloaded from the ISMN website. Note that - depending on when the data was downloaded - available stations and measurement values can vary slightly.

<img src="ismn.png" style="height: 450px;"/>

ISMN files are downloaded as a compressed `.zip` file after ordering the data from the website. It can then be extracted (with any zip software) locally into one (root) folder (in this case 'Data_separate_files') and will look like this:
```shell
Data_separate_files/
├── network/
│   ├── station/
│   │   ├── sensor.stm
│   │   ├── sensor.stm
│   │   ├── ...
│   │   ├── sensor.stm
│   │   ├── static_variables.csv
│   ├── station/
│   │   ├── ...
├── network/
│   ├── ...
├── ISMN_qualityflags_description.txt
├── Metadata.xml
├── Provider_qualityflags_description.txt
└── Readme.txt
```
You can either read from this extracted root folder, or read from the .zip file directly (the reader will then unzip files temporarily when needed). Reading from zip is therefore slower than reading the extracted files. Extracted files are (much) larger than compressed files.

The class for reading data from extracted or compressed files is `ismn.interface.ISMN_Interface`. It provides functions to access single networks, stations and sensors and the measured time series for each sensor as well as metadata for each station/sensor.

`ISMN_Interface` expects the path to the downloaded and locally stored ISMN data (zip-file or extracted root folder) as the only required argument.

In [ ]:
from ismn.interface import ISMN_Interface
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Either a .zip file or one folder that contains all networks
data_path = "/tmp/ismn/Data_separate_files_20090804_20201212.zip"
ismn_data = ISMN_Interface(data_path)

Files Processed:   0%|          | 0/110 [00:00<?, ?it/s]

Processing metadata for all ismn stations into folder /tmp/ismn/Data_separate_files_20090804_20201212.zip.
This may take a few minutes, but is only done once...
Hint: Use `parallel=True` to speed up metadata generation for large datasets


Files Processed: 100%|██████████| 110/110 [00:21<00:00,  5.14it/s]


Metadata generation finished after 21 Seconds.
Metadata and Log stored in /tmp/ismn/python_metadata


The first time you initialize `ISMN_Interface` for a dataset, it will collect metadata for all sensors in your data collection from various files (this is **always** done for **all** available networks). The program will iterate through **all** files and collect information such as station names, sensor time coverage, measurement depths, landcover/climate classes, soil properties etc. for each sensor. Depending on the number of files and whether zipped/extracted files are provided, this step can take a while. A log file is created in the displayed path. Parallel processing for this step can be activated manually by choosing `ISMN_Interface(data_path, parallel=True)` and will speed up metadata collection significantly. By default this step will create a folder called `python_metadata` (which contains the collected metadata as a `.csv` file) and place it inside the passed root directory. The next time the reader is created it will use `python_metadata` (if it is found) instead of generating it again (a different path to generate and search the metadata in can be passed via `ISMN_Interface(data_path, meta_path='/custom/meta/path')`.

**Note: When changing the data (e.g. if folders are added or removed from the collection that is passed to  the reader) make sure to delete the `python_metadata` folder and its content to force re-generating metadata. Otherwise data and metadata won't match anymore!!**

To limit reading to a selection of networks from the beginning (e.g. to load one network from a large collection, which is faster than loading all networks and then selecting one), their names can be passed to the reader (e.g. `ISMN_Interface(data_path, network=['REMEDHUS', 'SMOSMANIA'])`. Otherwise all networks stored under the given path are loaded. Limiting the number of networks when calling the reader will result in faster initialization, because less files are loaded (this does **not** affect metadata collection, which is always done for all networks in `data_path`).

You can call `ISMN_Interface(data_path, keep_loaded_data=True)`, which will cause that all sensor time series - once read - are kept in memory for faster subsequent calls. This can fill up your memory and is only recommended for small data collections, respectively if only a few networks are initialized.

Finally, you can define a custom temporary root folder, e.g. `ISMN_Interface(data_path, temp_root='/tmp/my_tempdata')`. By default this depends on your OS (e.g. `/tmp` for Linux), and is used by the reader to store some temporary files, e.g. when extracting while reading from `.zip`. The default `temp_root` is cleared automatically, so we recommend not to change this (unless e.g. because of access restrictions).

## ISMN Components (Overview)
Here we give a short overview over the components that build a ISMN data set. Each component provides various functions to work with its data. See the general docs for more details. This is just a basic overview of how to pick data out of the collection. In practice it is often required to loop over certain stations. This is described later in this tutorial. The components (hierarchically ordered) are `NetworkCollection`, `Network`, `Station`, `Sensor`.

### Collection
The `ISMN_Interface` object holds a 'collection' container for all loaded networks (e.g. 'FMI', 'REMEDHUS', ...). Each Network contains multiple Stations (e.g. 'SAA111', 'SA112', ... for the 'FMI' network). Each Station contains multiple Sensors (names not shown in this overview). You can access Networks directly from the reader, and subsequently access Stations and their Sensors.

In [ ]:
ismn_data.collection #  or just:  ismn_data  - overview over loaded networks and their stations

### Network

You can select a specific network from the collection above via its name. Here we pick the network 'SMOSMANIA' from our loaded set. Networks are sorted alphabetically, so you can also pass a number here, e.g. `ismn_data[4]` to get the fourth network from the list. This will (again) display all stations for that network.

In [ ]:
ismn_data.collection['SMOSMANIA']  #  overview over stations in SMOSMANIA network
                                   #  equivalent to ismn_data[4]

### Station

A network consists of multiple stations, multiple variables can be measured by different sensors at a station. You can select a specific station for a network via its name (stations are also sorted alphabetically and can be accessed by index). Here we access the Station 'SaintFelixdeLauragais' from the 'SMOSMANIA' network.

In [ ]:
ismn_data.collection['SMOSMANIA']['SaintFelixdeLauragais']  #  overview over sesnors at SaintFelixdeLauragais
                                                            #  equivalent to ismn_data[4][18]

Each station has a metadata attribute. The station metadata contains all metadata variables from all sensors that measure at the station (such as the sensor type, soil properties etc. per sensor). Therefore the station metadata can be different for different depths. You can call `MetaData` directly, or convert it to either a DataFrame (`MetaData.to_pd()`) or a dictionary (`MetaData.to_dict()`) of form:

```
{var_name: [(value, depth_from, depth_to), 
            ...], 
...}
```

In the example below, we read metadata without conversion. The first value in each Variable is the name of the metadata variable, the second the actual value for the variable. The third value is the depth range (depth_from, depth_to) that the value applies to - e.g. for soil properties (taken from the Harmonized World Soil Data Base) multiple layers are provided together with the ISMN data and during metadata generation the best matching depth for a sensor is selected. Some values apply to a specific depth (depth_from=depth_to) while others may apply to a depth range (usually depends on the network).

In [ ]:
ismn_data.collection['SMOSMANIA']['SaintFelixdeLauragais'].metadata  #  Get metadata for the station

### Sensor

Accessing sensors at a station works similar to accessing stations in a network. By default the name is created from the instrument type, the measured variable and the depth layer that the sensor operates in. Note that we can select a network from the data object directly without calling `.collection` first.

In [ ]:
ismn_data['SMOSMANIA']['SaintFelixdeLauragais']['ThetaProbe-ML2X_soil_moisture_0.050000_0.050000']
#  equivalent to ismn_data[4][18][4]

Each sensor has a measurement time series (access via `Sensor.read_data()`) and sensor specific metadata (via `Sensor.metadata`) assigned. Here we convert metadata to a data frame.

In [ ]:
sensor = ismn_data['SMOSMANIA']['SaintFelixdeLauragais']['ThetaProbe-ML2X_soil_moisture_0.050000_0.050000']
print(sensor.metadata.to_pd())
ts = sensor.read_data()
ax = ts.plot(figsize=(12,4))
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
display(ts)

Some metadata is different for each sensor (e.g. time series range), some depends on the location of the station and is therefore shared by multiple sensors at one station (landcover and climate classes etc.). Sometime metadata is missing (if not provided, indicated by 'unknown', or NaN). Some meta data depends on the depth of a sensor (e.g. soil properties), during metadata collection (in the beginning) these values were collected and assigned.

In [ ]:
sensor.metadata.to_dict()

## Some other important functions
Each component (network, station, sensor) contains different functions to handle its data. `ISMN_Interface` provides general functions to filter and iterate over its components and visualize them.

### Find nearest station
The data collection in `ISMN_Interface` contains a grid object that lists the locations of all **stations** in all active networks. For more details see https://github.com/TUW-GEO/pygeogrids

In [ ]:
import pandas as pd
grid = ismn_data.collection.grid
gpis, lons, lats = grid.get_grid_points()
pd.DataFrame(index=pd.Index(gpis, name='gpi'), 
             data={'lon': lons, 'lat': lats}).T

In [ ]:
# Using the GPI or coordinates, a station from **all** stations in **all** networks in the collection can be selected.
station, dist = ismn_data.collection.get_nearest_station(27.0, 68.0)
print(f'Station {station.name} is {int(dist)} metres away from the passed coordinates:')
assert ismn_data.collection.station4gpi(0) == station # same result when selecting with GPI
station

### Find network for a specific station
`ISMN_Interface` provides a function to find the network when only the name of a station is known. Here we simply read data for the first available sensor at the fist station in the found network.

In [ ]:
network = ismn_data.network_for_station('SAA111', name_only=False)
station = network[0]
sensor = station[0]
display(network, station, sensor)
sensor.read_data()

### Metadata overview
To get an overview over all metadata for the currently loaded networks, the attribute `ISMN_Interface.metadata` can be called. This will return a pandas DataFrame with all available metadata variables for different depths as (multi-index) columns and sensors as rows. The first column `idx` is the index of the sensor/filehandler, in line with what `ISMN_Interface.get_dataset_ids()` returns, and can therefore be used to read data via `ISMN_Interface.read()` as shown in the next section.

In [ ]:
ismn_data.metadata

 ### Selecting and reading specific sensors

We can filter the dataset a priori and get IDs of sensors that measure a specific variable. The ID can then be used to read the data directly. Here we extract the IDs of sensors in our data set that measure 'soil_temperature' in 0 to 1 meter depth and within a specific land cover and climate class (all conditions must be fulfilled).

In [ ]:
ids = ismn_data.get_dataset_ids(variable='soil_temperature', 
                                max_depth=1, 
                                filter_meta_dict={'lc_2005': 130, 'climate_KG': 'Csb'})
ids

Now we can use the so found IDs to read data from the according sensors.

In [ ]:
ts, meta = ismn_data.read(ids[1], return_meta=True)
meta
ax = ts.plot(figsize=(12,4), title=f'Time series for ID {ids[1]}')
ax.set_xlabel("Time [year]")
ax.set_ylabel("Soil Temp. [°C]")

### Plot station locations
We can define a similar query to plot station locations for a specific variable on a map. If a min/max depth is passed, only stations with a sensor that measures within the passed range are included. Note that this kind of visulisation needs additional (optional) packages installed (use `pip install ismn[plot]` to install them.)

In [ ]:
import cartopy.crs as ccrs
#plot available station on a map
fig, axs = plt.subplots(1, 2, figsize=(16,10), subplot_kw={'projection': ccrs.Robinson()})
ismn_data.plot_station_locations('soil_moisture', min_depth=0., max_depth=0.1, ax=axs[0])
ismn_data.plot_station_locations('soil_temperature', min_depth=0.5, ax=axs[1])
plt.show()


## Selecting and interating over sensors

It is often desired to iterate over all sensors that fulfill certain requirements (e.g. that measure soil moisture in a certain depth, and/or for a certain land cover class). For these cases the `collection` (and other components) provides iterators that take keywords and values for filtering the loaded networks/stations/sensor and iterating over the filtered time series (of a whole collection, a network, or a station).

### Select  by variable and depth
In this example we iterate over all sensors in the previously loaded collection (i.e. over all activated networks) that measure 'soil_moisture' in any depth (range) between 0 and 0.05 meters.

In [ ]:
for network, station, sensor in ismn_data.collection.iter_sensors(variable='soil_moisture', 
                                                                  depth=[0., 0.05]):
    display(network)
    display(station)
    display(sensor)

    data = sensor.read_data()

    print('\033[1m' + f'Metadata for sensor {sensor}:'+ '\033[0m')
    display(sensor.metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f'Time series for sensor {sensor.name}')
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
    break  # for this example we stop after the first sensor

### Selecting by variable and other metadata (1)
In this example we iterate over all sensors for the network 'RMSN' and filter out those that measure precipitation within an ESA CCI Landcover pixel that is marked as 'Cropland, rainfed' (10) or 'Grassland' (130). Available land cover classes to choose are:

In [ ]:
ismn_data.print_landcover_dict()

In [ ]:
for station, sensor in ismn_data['RSMN'].iter_sensors(variable='precipitation', 
                                                      filter_meta_dict={'lc_2010': [10, 130]}):
    display(station)
    display(sensor)
    
    data = sensor.read_data()
    metadata = sensor.metadata
    
    print('\033[1m' + f'Metadata for sensor {sensor}:' + '\033[0m')
    display(metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f'Time series for sensor {sensor.name}')
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Precipitation [mm]")
    break # for this example we stop after the first sensor

### Selecting by variable, depth and metadata (2)
In this example we iterate over all sensors in the collection and filter those that measure 'soil_moisture' between 0 and 10 cm within an ESA CCI Landcover pixel that is marked as 'Cropland, rainfed' (10) or 'Grassland' (130), and has one of the follwing climate classes assigned: 'Csc', 'Cfa', 'Dfc'. In addition we set all those soil moisture values that are **not** flagged as 'good' (G) to 'NaN'.

In [ ]:
ismn_data.print_climate_dict()

In [ ]:
from ismn.meta import Depth
for network, station, sensor in ismn_data.collection \
    .iter_sensors(variable='soil_moisture',
                  depth=Depth(0.,0.05),
                  filter_meta_dict={'lc_2010': [10, 130],
                                    'climate_KG':['Csc', 'Cfa', 'Dfc']}):
    
    display(network)
    display(station)
    display(sensor)
    
    data = sensor.read_data()
    data.loc[data['soil_moisture_flag'] != 'G', 'soil_moisture'] = np.nan
    metadata = sensor.metadata

    print('\033[1m' + f'Metadata for sensor {sensor}:'+ '\033[0m')
    display(metadata.to_pd())
    ax = data.plot(figsize=(12,4), title=f"G-flagged SM for '{sensor.name}' at station '{station.name}' in network '{network.name}''")
    ax.set_xlabel("Time [year]")
    ax.set_ylabel("Soil Moisture [$m^3 m^{-3}$]")
    break # for this example we stop after the first sensor